In [1]:
import simpy
import pygame
import random
import math

# ---------------- CONFIG ----------------
WIDTH, HEIGHT = 1000, 600
FPS = 60
DT = 0.1

NUM_PEOPLE = 40
BASE_SPEED = 1.2
PANIC_SPEED_MULT = 1.8

ALARM_TIME = 3          # seconds
FIRE_SPREAD_TIME = 2    # seconds
SMOKE_RADIUS = 80

EXIT_POSITIONS = [(950, 100), (950, 500)]

WALLS = [
    pygame.Rect(300, 0, 20, 400),
    pygame.Rect(600, 200, 20, 400),
]

# ----------------------------------------

def distance(a, b):
    return math.hypot(a[0] - b[0], a[1] - b[1])

class Fire:
    def __init__(self, env, x, y):
        self.env = env
        self.cells = [(x, y)]
        self.process = env.process(self.spread())

    def spread(self):
        while True:
            yield self.env.timeout(FIRE_SPREAD_TIME)
            x, y = random.choice(self.cells)
            nx = x + random.randint(-40, 40)
            ny = y + random.randint(-40, 40)
            nx = max(50, min(WIDTH - 50, nx))
            ny = max(50, min(HEIGHT - 50, ny))
            self.cells.append((nx, ny))

class Person:
    def __init__(self, env, x, y):
        self.env = env
        self.x = x
        self.y = y
        self.speed = BASE_SPEED * random.uniform(0.8, 1.2)
        self.panic = 0.0
        self.evacuated = False
        self.process = env.process(self.behavior())

    def behavior(self):
        while not self.evacuated:
            yield self.env.timeout(DT)

    def update(self, fire_cells, alarm_on):
        if self.evacuated or not alarm_on:
            return

        # Panic calculation
        for fx, fy in fire_cells:
            d = distance((self.x, self.y), (fx, fy))
            if d < SMOKE_RADIUS:
                self.panic = min(1.0, self.panic + 0.02)

        speed = self.speed * (1 + self.panic * PANIC_SPEED_MULT)

        # Move toward nearest exit
        exit_x, exit_y = min(EXIT_POSITIONS,
                             key=lambda e: distance((self.x, self.y), e))

        dx = exit_x - self.x
        dy = exit_y - self.y
        dist = math.hypot(dx, dy) or 1

        self.x += speed * dx / dist
        self.y += speed * dy / dist

        # Collision with walls
        person_rect = pygame.Rect(self.x - 4, self.y - 4, 8, 8)
        for wall in WALLS:
            if wall.colliderect(person_rect):
                self.x -= speed * dx / dist
                self.y -= speed * dy / dist

        # Exit reached
        if distance((self.x, self.y), (exit_x, exit_y)) < 10:
            self.evacuated = True

# ---------------- MAIN ----------------
env = simpy.Environment()

people = [
    Person(env,
           random.randint(50, 700),
           random.randint(50, 550))
    for _ in range(NUM_PEOPLE)
]

fire = Fire(env, 500, 300)
alarm_on = False

def alarm_process(env):
    global alarm_on
    yield env.timeout(ALARM_TIME)
    alarm_on = True

env.process(alarm_process(env))

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Evacuation Drill Simulation")
clock = pygame.time.Clock()

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    env.step()

    screen.fill((20, 20, 20))

    # Draw walls
    for wall in WALLS:
        pygame.draw.rect(screen, (100, 100, 100), wall)

    # Draw exits
    for ex in EXIT_POSITIONS:
        pygame.draw.circle(screen, (0, 255, 0), ex, 15)

    # Draw fire & smoke
    for fx, fy in fire.cells:
        pygame.draw.circle(screen, (255, 60, 60), (fx, fy), 8)
        pygame.draw.circle(screen, (80, 80, 80), (fx, fy), SMOKE_RADIUS, 1)

    # Update & draw people
    for p in people:
        p.update(fire.cells, alarm_on)

        if p.evacuated:
            continue

        if p.panic < 0.3:
            color = (100, 150, 255)
        elif p.panic < 0.7:
            color = (255, 200, 50)
        else:
            color = (255, 80, 80)

        pygame.draw.circle(screen, color, (int(p.x), int(p.y)), 5)

    # Alarm indicator
    if alarm_on:
        pygame.draw.rect(screen, (255, 0, 0), (0, 0, WIDTH, 8))

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
import xml.etree.ElementTree as ET
from svgpathtools import parse_path
 
# 1. Define Namespaces (Crucial for Inkscape)
ns = {
    'svg': 'http://www.w3.org/2000/svg',
    'inkscape': 'http://www.inkscape.org/namespaces/inkscape'
}
 
tree = ET.parse(r"C:\Users\joshu\Downloads\Fourth Plant Draft with refined layers Inkscape SVG.svg")
root = tree.getroot()
 
# 2. Iterate through groups that are actually Layers
for group in root.findall('.//svg:g[@inkscape:groupmode="layer"]', ns):
    # Get the layer name
    layer_name = group.get('{http://www.inkscape.org/namespaces/inkscape}label')
    print(f"--- Processing Layer: {layer_name} ---")
 
    # 3. Find paths ONLY inside this specific layer
    for path_elem in group.findall('svg:path', ns):
        d_string = path_elem.get('d')
        # 4. Convert to svgpathtools Path object here
        if d_string:
            path_obj = parse_path(d_string)
            print(f"   Found path with length: {path_obj.length()}")
            # Now you have the layer_name AND the path_obj together

--- Processing Layer: walls ---
   Found path with length: 275.70314
   Found path with length: 448.45702000000006
   Found path with length: 200.34766
   Found path with length: 200.34765999999996
   Found path with length: 448.46093999999994
   Found path with length: 275.6992
   Found path with length: 141.32029200000002
   Found path with length: 141.32032600000002
   Found path with length: 42.99609799999996
   Found path with length: 42.999998000000005
   Found path with length: 142.48045000000002
   Found path with length: 145.58984999999996
   Found path with length: 43.00000400000002
   Found path with length: 43.00000400000002
   Found path with length: 116.85937999999996
   Found path with length: 113.74999999999997
   Found path with length: 96.17969200000002
   Found path with length: 96.17578599999996
   Found path with length: 136.66405599999993
   Found path with length: 133.55079
   Found path with length: 49.441415999999975
   Found path with length: 49.44531800000001

In [5]:
"""Extract walls, obstacles, and exits layers into separate SVG files."""
 
import copy
import os
import xml.etree.ElementTree as ET
 
SRC = r"C:\Users\joshu\Downloads\Fourth Plant Draft with refined layers Inkscape SVG.svg"
OUT_NAME = "Fourth_Plant_{label}.svg"
 
NAMESPACES = {
    "svg": "http://www.w3.org/2000/svg",
    "ink": "http://www.inkscape.org/namespaces/inkscape",
    "sodipodi": "http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd",
}
 
# Ensure namespaces are preserved on output.
ET.register_namespace("", NAMESPACES["svg"])
ET.register_namespace("inkscape", NAMESPACES["ink"])
ET.register_namespace("sodipodi", NAMESPACES["sodipodi"])
 
tree = ET.parse(SRC)
root = tree.getroot()
 
 
def find_layer(label: str):
    """Return the <g> element for the given layer label."""
    for g in root.findall(".//svg:g", NAMESPACES):
        if (
            g.attrib.get(f"{{{NAMESPACES['ink']}}}groupmode") == "layer"
            and g.attrib.get(f"{{{NAMESPACES['ink']}}}label") == label
        ):
            return g
    return None
 
 
def build_root():
    """Create a new <svg> root mirroring the source root attributes."""
    return ET.Element(root.tag, root.attrib)
 
 
def write_layer(label: str):
    layer = find_layer(label)
    if layer is None:
        print(f"Layer '{label}' not found.")
        return
 
    new_root = build_root()
 
    defs = root.find("svg:defs", NAMESPACES)
    if defs is not None:
        new_root.append(copy.deepcopy(defs))
 
    new_root.append(copy.deepcopy(layer))
 
    out_path = os.path.join(os.path.dirname(SRC), OUT_NAME.format(label=label))
    ET.ElementTree(new_root).write(out_path, encoding="utf-8", xml_declaration=True)
    print(f"Wrote {out_path}")
 
 
if __name__ == "__main__":
    for layer_name in ("walls", "obstacles", "exit"):
        write_layer(layer_name)
 
 

Wrote C:\Users\joshu\Downloads\Fourth_Plant_walls.svg
Wrote C:\Users\joshu\Downloads\Fourth_Plant_obstacles.svg
Wrote C:\Users\joshu\Downloads\Fourth_Plant_exit.svg


In [8]:
WALL_IMG = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_walls.png"
OBST_IMG = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_obstacles.png"
EXIT_IMG = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_exit.png"


In [10]:
import numpy as np
def preview_mask(path):
    surf = pygame.image.load(path)
    arr = pygame.surfarray.array3d(surf)
    mask = np.mean(arr, axis=2) < 100
    print(path, "occupied:", np.sum(mask))


In [11]:
preview_mask(WALL_IMG)
preview_mask(OBST_IMG)
preview_mask(EXIT_IMG)


C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_walls.png occupied: 19793
C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_obstacles.png occupied: 242
C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_exit.png occupied: 3453


In [14]:
import pygame
import numpy as np
import random
from collections import deque

# --- CONFIGURATION ---
# Screen dimensions (The window size you will see)
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 800

# Logic Grid Size (Lower = Faster performance, Higher = More precise movement)
# We map the high-res images to this smaller grid for pathfinding calculations.
GRID_SIZE = 100 

# Agent Settings
NUM_AGENTS = 200
AGENT_SPEED = 0.5    # How fast they move in the grid
AGENT_SIZE = 3       # Visual size in pixels

# File Paths (Using Raw strings r"..." to handle backslashes)
WALL_IMG_PATH = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_walls.png"
OBST_IMG_PATH = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_obstacles.png"
EXIT_IMG_PATH = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_exit.png"

# Colors
COLOR_BG = (240, 240, 240)      # Light Grey Background
COLOR_WALL = (50, 50, 50)       # Dark Grey Walls
COLOR_OBSTACLE = (100, 100, 100)# Lighter Grey Obstacles
COLOR_EXIT = (50, 200, 50)      # Green Exits
COLOR_AGENT_SAFE = (0, 0, 255)  # Blue (Normal)
COLOR_AGENT_PANIC = (255, 0, 0) # Red (Panic)
COLOR_BTN_OFF = (0, 200, 0)
COLOR_BTN_ON = (200, 0, 0)

class SimulationMap:
    def __init__(self, w, h):
        self.w = w
        self.h = h
        self.walls = np.zeros((w, h), dtype=bool)
        self.exits = np.zeros((w, h), dtype=bool)
        # 9999 represents "Infinite distance" (unreachable)
        self.flow_field = np.full((w, h), 9999) 

    def load_and_process_images(self):
        """
        Loads the high-res PNGs, scales them to the simulation grid size,
        and creates boolean masks (True/False grids) for logic.
        """
        try:
            # 1. Load Images
            w_surf = pygame.image.load(WALL_IMG_PATH)
            o_surf = pygame.image.load(OBST_IMG_PATH)
            e_surf = pygame.image.load(EXIT_IMG_PATH)

            # 2. Scale to Logic Grid Size (e.g. 100x100) for fast calculation
            w_surf = pygame.transform.scale(w_surf, (self.w, self.h))
            o_surf = pygame.transform.scale(o_surf, (self.w, self.h))
            e_surf = pygame.transform.scale(e_surf, (self.w, self.h))

            # 3. Convert to arrays (Masking)
            # We look at pixel brightness. Dark pixels (< 150) are considered walls.
            w_arr = pygame.surfarray.array3d(w_surf)
            o_arr = pygame.surfarray.array3d(o_surf)
            e_arr = pygame.surfarray.array3d(e_surf)

            # Create Boolean Masks
            # "If the average color is dark, it's a wall"
            walls_mask = np.mean(w_arr, axis=2) < 200  
            obst_mask = np.mean(o_arr, axis=2) < 200
            
            # Combine Walls and Obstacles into one barrier map
            self.walls = np.logical_or(walls_mask, obst_mask)
            
            # Exits are also dark in the exit file
            self.exits = np.mean(e_arr, axis=2) < 200

            # IMPORTANT: Ensure Exits are not marked as Walls
            self.walls[self.exits] = False
            
            print("Maps loaded. Generating Navigation Mesh (this may take a moment)...")
            self._generate_flow_field()
            print("Navigation Mesh Ready.")

        except FileNotFoundError as e:
            print(f"ERROR: Could not find file. Check paths.\n{e}")
            pygame.quit()
            exit()

    def _generate_flow_field(self):
        """
        Breadth-First Search (BFS) to calculate distance from every cell to nearest exit.
        """
        queue = deque()
        
        # Initialize queue with all exit cells
        for x in range(self.w):
            for y in range(self.h):
                if self.exits[x, y]:
                    self.flow_field[x, y] = 0
                    queue.append((x, y))

        # Flood fill the grid
        while queue:
            x, y = queue.popleft()
            current_dist = self.flow_field[x, y]

            # Check all 8 neighbors
            neighbors = [
                (x+1, y), (x-1, y), (x, y+1), (x, y-1),
                (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1)
            ]

            for nx, ny in neighbors:
                # Check bounds
                if 0 <= nx < self.w and 0 <= ny < self.h:
                    # If not a wall and we found a shorter path to it
                    if not self.walls[nx, ny]:
                        if self.flow_field[nx, ny] > current_dist + 1:
                            self.flow_field[nx, ny] = current_dist + 1
                            queue.append((nx, ny))

class Agent:
    def __init__(self, x, y):
        self.pos = np.array([float(x), float(y)])
        self.vel = np.array([0.0, 0.0])
        self.escaped = False
        self.panic = False

    def update(self, sim_map):
        if self.escaped: return

        # Get current integer grid position
        px, py = int(self.pos[0]), int(self.pos[1])

        # Check if Escaped
        if sim_map.exits[px, py]:
            self.escaped = True
            return

        if self.panic:
            # --- EVACUATION LOGIC ---
            # Look at neighbors and move to the one with the smallest flow_field value
            best_dist = sim_map.flow_field[px, py]
            target = self.pos
            
            found_better = False
            # Scan 3x3 area
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    nx, ny = px + dx, py + dy
                    if 0 <= nx < sim_map.w and 0 <= ny < sim_map.h:
                        dist = sim_map.flow_field[nx, ny]
                        if dist < best_dist:
                            best_dist = dist
                            # Add 0.5 to move to center of cell
                            target = np.array([nx + 0.5, ny + 0.5])
                            found_better = True
            
            if found_better:
                # Calculate direction vector
                direction = target - self.pos
                norm = np.linalg.norm(direction)
                if norm > 0:
                    self.vel = (direction / norm) * AGENT_SPEED
            else:
                # Stuck or local minimum (wiggle to get free)
                self.vel = np.array([random.uniform(-1, 1), random.uniform(-1, 1)]) * 0.1

        else:
            # --- IDLE LOGIC (Brownian Motion) ---
            if random.random() < 0.1: # Change direction occasionally
                self.vel = np.array([random.uniform(-0.2, 0.2), random.uniform(-0.2, 0.2)])

        # Move
        new_pos = self.pos + self.vel
        
        # Collision Check (Walls)
        npx, npy = int(new_pos[0]), int(new_pos[1])
        if 0 <= npx < sim_map.w and 0 <= npy < sim_map.h:
            if not sim_map.walls[npx, npy]:
                self.pos = new_pos
            else:
                self.vel *= -0.5 # Bounce

def main():
    pygame.init()
    screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
    pygame.display.set_caption("Fire Evacuation Simulator")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("Arial", 20)

    # 1. Setup Map
    sim_map = SimulationMap(GRID_SIZE, GRID_SIZE)
    sim_map.load_and_process_images()

    # 2. Scale factors (To draw the small grid onto the big screen)
    scale_x = WINDOW_WIDTH / GRID_SIZE
    scale_y = WINDOW_HEIGHT / GRID_SIZE

    # 3. Spawn Agents
    agents = []
    while len(agents) < NUM_AGENTS:
        rx = random.randint(1, GRID_SIZE-2)
        ry = random.randint(1, GRID_SIZE-2)
        if not sim_map.walls[rx, ry]:
            agents.append(Agent(rx, ry))

    # UI Button
    fire_active = False
    btn_rect = pygame.Rect(WINDOW_WIDTH - 150, 20, 130, 40)

    running = True
    while running:
        # --- INPUT ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    # Check Button Click
                    if btn_rect.collidepoint(event.pos):
                        fire_active = not fire_active
                        # Wake up agents
                        for a in agents: a.panic = fire_active

        # --- UPDATE ---
        for a in agents:
            a.update(sim_map)

        # --- DRAW ---
        screen.fill(COLOR_BG)

        # Draw Map (Iterate grid)
        # Note: For better performance with static maps, we would usually blit a pre-rendered surface.
        # But drawing rectangles is fine for this grid size.
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                rect = (x * scale_x, y * scale_y, scale_x + 1, scale_y + 1)
                if sim_map.walls[x, y]:
                    pygame.draw.rect(screen, COLOR_WALL, rect)
                elif sim_map.exits[x, y]:
                    pygame.draw.rect(screen, COLOR_EXIT, rect)

        # Draw Agents
        for a in agents:
            if a.escaped: continue
            color = COLOR_AGENT_PANIC if a.panic else COLOR_AGENT_SAFE
            # Scale position to screen
            sx = int(a.pos[0] * scale_x)
            sy = int(a.pos[1] * scale_y)
            pygame.draw.circle(screen, color, (sx, sy), AGENT_SIZE)

        # Draw UI
        pygame.draw.rect(screen, COLOR_BTN_ON if fire_active else COLOR_BTN_OFF, btn_rect)
        pygame.draw.rect(screen, (0,0,0), btn_rect, 2) # Border
        
        lbl = "STOP FIRE" if fire_active else "START FIRE"
        txt = font.render(lbl, True, (255,255,255))
        screen.blit(txt, (btn_rect.x + 15, btn_rect.y + 10))

        # Stats
        escaped_count = sum(1 for a in agents if a.escaped)
        stat_txt = font.render(f"Escaped: {escaped_count}/{NUM_AGENTS}", True, (0,0,0))
        screen.blit(stat_txt, (10, 10))

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

if __name__ == "__main__":
    main()

Maps loaded. Generating Navigation Mesh (this may take a moment)...
Navigation Mesh Ready.


In [22]:
import pygame
import numpy as np
import random
from collections import deque

# --- CONFIGURATION ---
# We no longer hardcode window size. It will adapt to your image.
MAX_WINDOW_SIZE = 1000  # Max width or height in pixels
GRID_SIZE = 120       # Logic resolution (keep this 100-150 for speed)

# Agent Settings
NUM_AGENTS = 150
AGENT_SPEED = 0.5
AGENT_RADIUS = 3

# File Paths
WALL_IMG_PATH = r"C:\Users\joshu\Downloads\Fifth Plant Draft _walls.png"
EXIT_IMG_PATH = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_exit.png"

# Colors
COLOR_AGENT_SAFE = (0, 100, 255) # Blue
COLOR_AGENT_PANIC = (255, 50, 50)# Red
COLOR_FIRE_CORE = (255, 69, 0)
COLOR_FIRE_OUTER = (255, 140, 0)

class SimulationMap:
    def __init__(self):
        self.w = GRID_SIZE
        self.h = GRID_SIZE
        self.walls = None
        self.exits = None
        self.flow_field = None
        
        # Visual assets
        self.bg_surface = None
        self.screen_w = 0
        self.screen_h = 0

    def load_and_setup(self):
        try:
            # 1. Load Original High-Res Images
            raw_wall = pygame.image.load(WALL_IMG_PATH)
            raw_exit = pygame.image.load(EXIT_IMG_PATH)

            # 2. Calculate Aspect Ratio & Screen Size
            img_w, img_h = raw_wall.get_size()
            scale_factor = min(MAX_WINDOW_SIZE / img_w, MAX_WINDOW_SIZE / img_h)
            
            self.screen_w = int(img_w * scale_factor)
            self.screen_h = int(img_h * scale_factor)

            # 3. Create the Visual Background (High Quality)
            # Scale images to fit the screen dimensions exactly
            vis_wall = pygame.transform.scale(raw_wall, (self.screen_w, self.screen_h))
            vis_exit = pygame.transform.scale(raw_exit, (self.screen_w, self.screen_h))
            
            # Combine them onto one background surface for drawing
            self.bg_surface = pygame.Surface((self.screen_w, self.screen_h))
            self.bg_surface.fill((255, 255, 255)) # White base
            self.bg_surface.blit(vis_wall, (0, 0))
            # Blend exit (optional, just to make sure it's visible)
            vis_exit.set_colorkey((255,255,255)) # Assuming white background in exit png
            self.bg_surface.blit(vis_exit, (0, 0))

            # 4. Create the Logic Grid (Low Res for Math)
            # We squash the image down to GRID_SIZE (e.g., 100x100) just for calculations
            logic_wall = pygame.transform.scale(raw_wall, (self.w, self.h))
            logic_exit = pygame.transform.scale(raw_exit, (self.w, self.h))

            wall_arr = pygame.surfarray.array3d(logic_wall)
            exit_arr = pygame.surfarray.array3d(logic_exit)

            # Create Masks (Dark pixels = features)
            self.walls = np.mean(wall_arr, axis=2) < 200
            self.exits = np.mean(exit_arr, axis=2) < 200
            self.walls[self.exits] = False # Exits aren't walls

            # Adjust Grid Aspect Ratio if image isn't square
            # (We keep logic grid square 100x100 for simplicity, but map coordinates carefully later)
            
            print(f"Screen set to: {self.screen_w}x{self.screen_h}")
            print("Generating navigation paths...")
            self._generate_flow_field()
            print("Ready.")

        except FileNotFoundError as e:
            print(f"Error: {e}")
            exit()

    def _generate_flow_field(self):
        """Standard BFS for pathfinding."""
        self.flow_field = np.full((self.w, self.h), 9999)
        queue = deque()
        
        for x in range(self.w):
            for y in range(self.h):
                if self.exits[x, y]:
                    self.flow_field[x, y] = 0
                    queue.append((x, y))

        while queue:
            x, y = queue.popleft()
            current = self.flow_field[x, y]
            neighbors = [(x+1,y), (x-1,y), (x,y+1), (x,y-1), (x+1,y+1), (x-1,y-1)]
            
            for nx, ny in neighbors:
                if 0 <= nx < self.w and 0 <= ny < self.h:
                    if not self.walls[nx, ny]:
                        if self.flow_field[nx, ny] > current + 1:
                            self.flow_field[nx, ny] = current + 1
                            queue.append((nx, ny))

class FireManager:
    def __init__(self):
        self.particles = []

    def start_fire(self, sim_map):
        self.particles = []
        for _ in range(15):
            rx, ry = random.randint(0, sim_map.w-1), random.randint(0, sim_map.h-1)
            if not sim_map.walls[rx, ry]:
                self.particles.append([rx, ry, random.uniform(2, 5), random.uniform(8, 15)])

    def update(self):
        for p in self.particles:
            p[2] = p[2] + 0.2 if p[2] < p[3] else random.uniform(2, 5)

    def draw(self, screen, scale_x, scale_y):
        for p in self.particles:
            sx = int(p[0] * scale_x)
            sy = int(p[1] * scale_y)
            pygame.draw.circle(screen, COLOR_FIRE_OUTER, (sx, sy), int(p[2] * scale_x))
            pygame.draw.circle(screen, COLOR_FIRE_CORE, (sx, sy), int(p[2] * scale_x * 0.6))

class Agent:
    def __init__(self, x, y):
        self.pos = np.array([float(x), float(y)]) # Logic coordinates (0-100)
        self.vel = np.array([0.0, 0.0])
        self.escaped = False
        self.panic = False

    def update(self, sim_map):
        if self.escaped: return
        px, py = int(self.pos[0]), int(self.pos[1])

        # Bounds check
        if px < 0 or px >= sim_map.w or py < 0 or py >= sim_map.h:
            return 

        if sim_map.exits[px, py]:
            self.escaped = True
            return

        if self.panic:
            best = sim_map.flow_field[px, py]
            target = self.pos
            found = False
            for dx in [-1,0,1]:
                for dy in [-1,0,1]:
                    nx, ny = px+dx, py+dy
                    if 0<=nx<sim_map.w and 0<=ny<sim_map.h:
                        if sim_map.flow_field[nx,ny] < best:
                            best = sim_map.flow_field[nx,ny]
                            target = np.array([nx+0.5, ny+0.5])
                            found = True
            if found:
                direction = target - self.pos
                norm = np.linalg.norm(direction)
                if norm > 0: self.vel = (direction/norm) * AGENT_SPEED
        else:
            if random.random() < 0.1:
                self.vel = np.array([random.uniform(-0.5,0.5), random.uniform(-0.5,0.5)])

        # Collision & Move
        new_pos = self.pos + self.vel
        npx, npy = int(new_pos[0]), int(new_pos[1])
        if 0<=npx<sim_map.w and 0<=npy<sim_map.h:
            if not sim_map.walls[npx, npy]:
                self.pos = new_pos
            else:
                self.vel *= -0.5

def main():
    pygame.init()
    
    # 1. Initialize Map Logic First to get Dimensions
    sim_map = SimulationMap()
    sim_map.load_and_setup()

    # 2. Setup Screen based on Image Dimensions
    screen = pygame.display.set_mode((sim_map.screen_w, sim_map.screen_h))
    pygame.display.set_caption("Fire Evacuation Sim")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("Arial", 22, bold=True)
    
    fire_mgr = FireManager()

    # Calculate scaling (Logic Grid -> Screen Pixels)
    scale_x = sim_map.screen_w / GRID_SIZE
    scale_y = sim_map.screen_h / GRID_SIZE

    # Spawn Agents
    agents = []
    while len(agents) < NUM_AGENTS:
        rx, ry = random.randint(1, GRID_SIZE-2), random.randint(1, GRID_SIZE-2)
        if not sim_map.walls[rx, ry] and not sim_map.exits[rx, ry]:
            agents.append(Agent(rx, ry))

    # UI
    fire_active = False
    btn_rect = pygame.Rect(sim_map.screen_w - 140, 20, 120, 40)

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT: running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1 and btn_rect.collidepoint(event.pos):
                    fire_active = not fire_active
                    for a in agents: a.panic = fire_active
                    if fire_active: fire_mgr.start_fire(sim_map)
                    else: fire_mgr.particles = []

        # Update
        for a in agents: a.update(sim_map)
        if fire_active: fire_mgr.update()

        # Draw
        # A. Draw the high-quality background image
        screen.blit(sim_map.bg_surface, (0, 0))

        # B. Draw Fire
        if fire_active: fire_mgr.draw(screen, scale_x, scale_y)

        # C. Draw Agents
        for a in agents:
            if a.escaped: continue
            col = COLOR_AGENT_PANIC if a.panic else COLOR_AGENT_SAFE
            # Transform logic pos (0-100) to screen pos (0-Width)
            sx = int(a.pos[0] * scale_x)
            sy = int(a.pos[1] * scale_y)
            pygame.draw.circle(screen, col, (sx, sy), AGENT_RADIUS)

        # D. UI
        col = COLOR_FIRE_CORE if fire_active else (50, 200, 50)
        pygame.draw.rect(screen, col, btn_rect, border_radius=5)
        txt = font.render("STOP" if fire_active else "FIRE", True, (255,255,255))
        screen.blit(txt, (btn_rect.x + 35, btn_rect.y + 7))

        pygame.display.flip()
        clock.tick(30)

    pygame.quit()

if __name__ == "__main__":
    main()

Screen set to: 1000x751
Generating navigation paths...
Ready.


In [37]:
import pygame
import numpy as np
import random
from collections import deque

# --- CONFIGURATION ---
MAX_WINDOW_SIZE = 1000
GRID_SIZE = 120   # Logic resolution
FPS = 60

# Simulation Settings
NUM_AGENTS = 200
AGENT_RADIUS = 3
FIRE_SPREAD_SPEED = 0.05  # Chance per frame for fire to spread
FIRE_KILL_RADIUS = 5      # Distance (in logic grid) to kill agents

# File Paths (Using your paths)
WALL_IMG_PATH = r"C:\Users\joshu\Downloads\Fifth Plant Draft _walls.png"
EXIT_IMG_PATH = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_exit.png"

# Colors
COL_AGENT_SAFE = (0, 120, 255)
COL_AGENT_PANIC = (255, 50, 50)
COL_SMOKE = (50, 50, 50)
COL_DEAD = (20, 20, 20)

class SimulationMap:
    def __init__(self):
        self.w = GRID_SIZE
        self.h = GRID_SIZE
        self.walls = None
        self.exits = None
        self.flow_field = None
        self.screen_w = 0
        self.screen_h = 0
        self.bg_surface = None

    def load_and_setup(self):
        try:
            raw_wall = pygame.image.load(WALL_IMG_PATH)
            raw_exit = pygame.image.load(EXIT_IMG_PATH)

            img_w, img_h = raw_wall.get_size()
            scale = min(MAX_WINDOW_SIZE / img_w, MAX_WINDOW_SIZE / img_h)
            self.screen_w = int(img_w * scale)
            self.screen_h = int(img_h * scale)

            # Create High-Quality Background
            self.bg_surface = pygame.Surface((self.screen_w, self.screen_h))
            self.bg_surface.fill((250, 250, 250)) 
            vis_wall = pygame.transform.scale(raw_wall, (self.screen_w, self.screen_h))
            vis_exit = pygame.transform.scale(raw_exit, (self.screen_w, self.screen_h))
            self.bg_surface.blit(vis_wall, (0,0))
            vis_exit.set_colorkey((255,255,255))
            self.bg_surface.blit(vis_exit, (0,0))

            # Logic Grids
            logic_wall = pygame.transform.scale(raw_wall, (self.w, self.h))
            logic_exit = pygame.transform.scale(raw_exit, (self.w, self.h))
            w_arr = pygame.surfarray.array3d(logic_wall)
            e_arr = pygame.surfarray.array3d(logic_exit)

            self.walls = np.mean(w_arr, axis=2) < 200
            self.exits = np.mean(e_arr, axis=2) < 200
            self.walls[self.exits] = False 

            print("Generating navigation mesh...")
            self._generate_flow_field()
            print("Ready.")

        except Exception as e:
            print(f"Error: {e}")
            exit()

    def _generate_flow_field(self):
        self.flow_field = np.full((self.w, self.h), 9999)
        queue = deque()
        for x in range(self.w):
            for y in range(self.h):
                if self.exits[x, y]:
                    self.flow_field[x, y] = 0
                    queue.append((x, y))

        while queue:
            x, y = queue.popleft()
            val = self.flow_field[x, y]
            neighbors = [(x+1,y),(x-1,y),(x,y+1),(x,y-1),(x+1,y+1),(x-1,y-1)]
            for nx, ny in neighbors:
                if 0<=nx<self.w and 0<=ny<self.h:
                    if not self.walls[nx, ny] and self.flow_field[nx, ny] > val+1:
                        self.flow_field[nx, ny] = val+1
                        queue.append((nx, ny))

# --- NEW VISUAL EFFECTS SYSTEMS ---

class ParticleSystem:
    """Handles Smoke and Spark effects."""
    def __init__(self):
        self.smoke = [] # [x, y, radius, alpha]

    def add_smoke(self, x, y):
        if random.random() < 0.3: # Don't spawn every frame
            # Spawn slightly offset
            off_x = random.uniform(-1, 1)
            off_y = random.uniform(-1, 1)
            self.smoke.append([x + off_x, y + off_y, random.uniform(2, 5), 150])

    def update(self):
        # Update Smoke
        for s in self.smoke:
            s[1] -= 0.1  # Rise up
            s[2] += 0.05 # Expand
            s[3] -= 2    # Fade out
        # Remove invisible smoke
        self.smoke = [s for s in self.smoke if s[3] > 0]

    def draw(self, screen, scale_x, scale_y):
        # We use a temporary surface to handle alpha (transparency) efficiently
        surf = pygame.Surface(screen.get_size(), pygame.SRCALPHA)
        for s in self.smoke:
            sx, sy = int(s[0]*scale_x), int(s[1]*scale_y)
            sr = int(s[2]*scale_x)
            if sr > 0:
                pygame.draw.circle(surf, (*COL_SMOKE, int(s[3])), (sx, sy), sr)
        screen.blit(surf, (0,0))

class FireManager:
    """Handles spreading fire logic."""
    def __init__(self):
        self.fire_cells = [] # List of [x, y] logic coordinates
        self.grid_status = np.zeros((GRID_SIZE, GRID_SIZE), dtype=bool) 
        self.start_time = 0

    def ignite(self, sim_map):
        self.fire_cells = []
        self.grid_status.fill(False)
        # Start 3 fires in random spots
        for _ in range(3):
            rx, ry = random.randint(1, sim_map.w-2), random.randint(1, sim_map.h-2)
            if not sim_map.walls[rx, ry]:
                self.fire_cells.append([rx, ry])
                self.grid_status[rx, ry] = True

    def update(self, sim_map, particle_sys):
        # 1. Spread Fire
        new_fire = []
        for fx, fy in self.fire_cells:
            # Chance to spread to neighbor
            if random.random() < FIRE_SPREAD_SPEED:
                dx, dy = random.randint(-1, 1), random.randint(-1, 1)
                nx, ny = fx+dx, fy+dy
                if 0 <= nx < sim_map.w and 0 <= ny < sim_map.h:
                    if not sim_map.walls[nx, ny] and not self.grid_status[nx, ny]:
                        new_fire.append([nx, ny])
                        self.grid_status[nx, ny] = True
            
            # Generate Smoke at this fire location
            particle_sys.add_smoke(fx, fy)

        self.fire_cells.extend(new_fire)

    def draw(self, screen, scale_x, scale_y):
        # Draw Fire with an "Additive" glow effect
        surf = pygame.Surface(screen.get_size(), pygame.SRCALPHA)
        
        # Draw outer glow
        for fx, fy in self.fire_cells:
            sx, sy = int(fx*scale_x), int(fy*scale_y)
            # Jitter size for flickering effect
            r_outer = int((scale_x * 1.5) + random.uniform(-2, 2))
            r_inner = int((scale_x * 0.8) + random.uniform(-1, 1))
            
            # Orange Glow
            pygame.draw.circle(surf, (255, 100, 0, 100), (sx, sy), r_outer)
            # Yellow Core
            pygame.draw.circle(surf, (255, 255, 100, 200), (sx, sy), r_inner)
            
        screen.blit(surf, (0,0), special_flags=pygame.BLEND_ADD)

class Agent:
    def __init__(self, x, y):
        self.pos = np.array([float(x), float(y)])
        # Randomize speed for realism (Young vs Old vs Panic)
        self.speed = random.uniform(0.3, 0.7) 
        self.vel = np.array([0.0, 0.0])
        self.escaped = False
        self.panic = False
        self.dead = False

    def update(self, sim_map, fire_mgr):
        if self.escaped or self.dead: return

        px, py = int(self.pos[0]), int(self.pos[1])

        # 1. Check Death (Touched Fire)
        if 0 <= px < sim_map.w and 0 <= py < sim_map.h:
            if fire_mgr.grid_status[px, py]:
                self.dead = True
                return

        # 2. Check Exit
        if sim_map.exits[px, py]:
            self.escaped = True
            return

        # 3. Movement Logic
        if self.panic:
            # Flow Field Navigation
            best = sim_map.flow_field[px, py]
            target = self.pos
            found = False
            
            # Look for neighbor closer to exit
            for dx in [-1,0,1]:
                for dy in [-1,0,1]:
                    nx, ny = px+dx, py+dy
                    if 0<=nx<sim_map.w and 0<=ny<sim_map.h:
                        dist = sim_map.flow_field[nx, ny]
                        if dist < best:
                            best = dist
                            # Add random jitter so they don't walk in perfect lines
                            target = np.array([nx + 0.5 + random.uniform(-0.1, 0.1), 
                                               ny + 0.5 + random.uniform(-0.1, 0.1)])
                            found = True
            
            if found:
                direction = target - self.pos
                norm = np.linalg.norm(direction)
                if norm > 0: self.vel = (direction/norm) * self.speed
            else:
                # Trapped?
                self.vel = np.array([0.0, 0.0])
        else:
            # Idle Wander
            if random.random() < 0.05:
                self.vel = np.array([random.uniform(-0.5,0.5), random.uniform(-0.5,0.5)]) * 0.5

        # 4. Apply Physics
        new_pos = self.pos + self.vel
        npx, npy = int(new_pos[0]), int(new_pos[1])
        
        # Wall Collision
        if 0 <= npx < sim_map.w and 0 <= npy < sim_map.h:
            if not sim_map.walls[npx, npy]:
                self.pos = new_pos
            else:
                self.vel *= -0.5 # Bounce

def main():
    pygame.init()
    sim_map = SimulationMap()
    sim_map.load_and_setup()

    screen = pygame.display.set_mode((sim_map.screen_w, sim_map.screen_h))
    pygame.display.set_caption("Advanced Evacuation Simulator")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("Segoe UI", 24, bold=True)
    
    # Systems
    fire_mgr = FireManager()
    particles = ParticleSystem()

    scale_x = sim_map.screen_w / GRID_SIZE
    scale_y = sim_map.screen_h / GRID_SIZE

    # Spawn Agents
    agents = []
    while len(agents) < NUM_AGENTS:
        rx, ry = random.randint(1, GRID_SIZE-2), random.randint(1, GRID_SIZE-2)
        if not sim_map.walls[rx, ry] and not sim_map.exits[rx, ry]:
            agents.append(Agent(rx, ry))

    # UI State
    fire_active = False
    btn_rect = pygame.Rect(sim_map.screen_w // 2 - 75, 20, 150, 50)

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT: running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1 and btn_rect.collidepoint(event.pos):
                    fire_active = not fire_active
                    for a in agents: a.panic = fire_active
                    if fire_active: fire_mgr.ignite(sim_map)
                    else: 
                        fire_mgr.fire_cells = []
                        fire_mgr.grid_status.fill(False)
                        particles.smoke = []

        # --- UPDATES ---
        if fire_active:
            fire_mgr.update(sim_map, particles)
            particles.update()

        for a in agents:
            a.update(sim_map, fire_mgr)

        # --- DRAWING ---
        # 1. Background
        screen.blit(sim_map.bg_surface, (0, 0))

        # 2. Dead Agents (Black X)
        for a in agents:
            if a.dead:
                sx, sy = int(a.pos[0]*scale_x), int(a.pos[1]*scale_y)
                # Draw a little cross
                pygame.draw.line(screen, COL_DEAD, (sx-3, sy-3), (sx+3, sy+3), 2)
                pygame.draw.line(screen, COL_DEAD, (sx+3, sy-3), (sx-3, sy+3), 2)

        # 3. Live Agents
        for a in agents:
            if a.escaped or a.dead: continue
            col = COL_AGENT_PANIC if a.panic else COL_AGENT_SAFE
            sx, sy = int(a.pos[0]*scale_x), int(a.pos[1]*scale_y)
            pygame.draw.circle(screen, col, (sx, sy), AGENT_RADIUS)

        # 4. Smoke & Fire (Draw on top of agents to simulate depth)
        particles.draw(screen, scale_x, scale_y)
        if fire_active: 
            fire_mgr.draw(screen, scale_x, scale_y)

        # 5. UI Overlay
        # Transparent stats bar
        overlay = pygame.Surface((200, 90), pygame.SRCALPHA)
        overlay.fill((0, 0, 0, 180))
        screen.blit(overlay, (10, 10))
        
        escaped = sum(1 for a in agents if a.escaped)
        died = sum(1 for a in agents if a.dead)
        alive = NUM_AGENTS - escaped - died
        
        screen.blit(font.render(f"Escaped: {escaped}", True, (50, 255, 50)), (20, 20))
        screen.blit(font.render(f"Alive:   {alive}", True, (100, 200, 255)), (20, 45))
        screen.blit(font.render(f"Casualties: {died}", True, (255, 50, 50)), (20, 70))

        # Button
        btn_col = (220, 60, 60) if fire_active else (60, 220, 60)
        pygame.draw.rect(screen, btn_col, btn_rect, border_radius=8)
        pygame.draw.rect(screen, (255,255,255), btn_rect, 2, border_radius=8)
        
        lbl = "EVACUATE!" if not fire_active else "RESET"
        txt = font.render(lbl, True, (255,255,255))
        txt_rect = txt.get_rect(center=btn_rect.center)
        screen.blit(txt, txt_rect)

        pygame.display.flip()
        clock.tick(FPS)

    pygame.quit()

if __name__ == "__main__":
    main()

Generating navigation mesh...
Ready.


In [38]:
import pygame
import numpy as np
import random
from collections import deque
from scipy.ndimage import binary_dilation  # <--- REQUIRED for wall thickening

# --- CONFIGURATION ---
MAX_WINDOW_SIZE = 1000
GRID_SIZE = 120   # Logic resolution
FPS = 60

# Simulation Settings
NUM_AGENTS = 200
AGENT_RADIUS = 3
FIRE_SPREAD_SPEED = 0.05
FIRE_KILL_RADIUS = 5

# File Paths
# Update these paths if necessary
WALL_IMG_PATH = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_walls.png"
EXIT_IMG_PATH = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_exit.png"
# Note: If you have an obstacles file, add it here too, otherwise walls handles it.

# Colors
COL_AGENT_SAFE = (0, 120, 255)
COL_AGENT_PANIC = (255, 50, 50)
COL_SMOKE = (50, 50, 50)
COL_DEAD = (20, 20, 20)

class SimulationMap:
    def __init__(self):
        self.w = GRID_SIZE
        self.h = GRID_SIZE
        self.walls = None
        self.exits = None
        self.flow_field = None
        self.screen_w = 0
        self.screen_h = 0
        self.bg_surface = None

    def load_and_setup(self):
        try:
            # Load images
            raw_wall = pygame.image.load(WALL_IMG_PATH)
            raw_exit = pygame.image.load(EXIT_IMG_PATH)

            img_w, img_h = raw_wall.get_size()
            scale = min(MAX_WINDOW_SIZE / img_w, MAX_WINDOW_SIZE / img_h)
            self.screen_w = int(img_w * scale)
            self.screen_h = int(img_h * scale)

            # Create Background
            self.bg_surface = pygame.Surface((self.screen_w, self.screen_h))
            self.bg_surface.fill((250, 250, 250)) 
            vis_wall = pygame.transform.scale(raw_wall, (self.screen_w, self.screen_h))
            vis_exit = pygame.transform.scale(raw_exit, (self.screen_w, self.screen_h))
            
            # Blit
            self.bg_surface.blit(vis_wall, (0,0))
            vis_exit.set_colorkey((255,255,255))
            self.bg_surface.blit(vis_exit, (0,0))

            # Logic Grids
            logic_wall = pygame.transform.scale(raw_wall, (self.w, self.h))
            logic_exit = pygame.transform.scale(raw_exit, (self.w, self.h))
            w_arr = pygame.surfarray.array3d(logic_wall)
            e_arr = pygame.surfarray.array3d(logic_exit)

            # Thresholding logic
            self.walls = np.mean(w_arr, axis=2) < 220
            self.exits = np.mean(e_arr, axis=2) < 220
            
            # --- FIX 1: THICKEN WALLS ---
            # This fills in diagonal gaps and makes walls "watertight"
            print("Thickening walls...")
            self.walls = binary_dilation(self.walls, iterations=1)

            # Ensure exits are carved out of walls
            self.walls[self.exits] = False 

            print("Generating navigation mesh...")
            self._generate_flow_field()
            print("Ready.")

        except Exception as e:
            print(f"Error: {e}")
            exit()

    def _generate_flow_field(self):
        self.flow_field = np.full((self.w, self.h), 9999)
        queue = deque()
        for x in range(self.w):
            for y in range(self.h):
                if self.exits[x, y]:
                    self.flow_field[x, y] = 0
                    queue.append((x, y))

        while queue:
            x, y = queue.popleft()
            val = self.flow_field[x, y]
            # 8-Way Movement
            neighbors = [(x+1,y),(x-1,y),(x,y+1),(x,y-1),(x+1,y+1),(x-1,y-1),(x+1,y-1),(x-1,y+1)]
            for nx, ny in neighbors:
                if 0<=nx<self.w and 0<=ny<self.h:
                    if not self.walls[nx, ny]:
                        # Diagonal movement costs slightly more (1.4) to discourage zig-zags
                        cost = 1 if (nx==x or ny==y) else 1.4
                        if self.flow_field[nx, ny] > val + cost:
                            self.flow_field[nx, ny] = val + cost
                            queue.append((nx, ny))

# --- VISUAL EFFECTS ---

class ParticleSystem:
    def __init__(self):
        self.smoke = [] 
    def add_smoke(self, x, y):
        if random.random() < 0.3:
            self.smoke.append([x + random.uniform(-1, 1), y + random.uniform(-1, 1), random.uniform(2, 5), 150])
    def update(self):
        for s in self.smoke:
            s[1] -= 0.1; s[2] += 0.05; s[3] -= 2
        self.smoke = [s for s in self.smoke if s[3] > 0]
    def draw(self, screen, scale_x, scale_y):
        surf = pygame.Surface(screen.get_size(), pygame.SRCALPHA)
        for s in self.smoke:
            sx, sy = int(s[0]*scale_x), int(s[1]*scale_y)
            sr = int(s[2]*scale_x)
            if sr > 0: pygame.draw.circle(surf, (*COL_SMOKE, int(s[3])), (sx, sy), sr)
        screen.blit(surf, (0,0))

class FireManager:
    def __init__(self):
        self.fire_cells = []; self.grid_status = np.zeros((GRID_SIZE, GRID_SIZE), dtype=bool) 
    def ignite(self, sim_map):
        self.fire_cells = []; self.grid_status.fill(False)
        for _ in range(3):
            rx, ry = random.randint(1, sim_map.w-2), random.randint(1, sim_map.h-2)
            if not sim_map.walls[rx, ry]:
                self.fire_cells.append([rx, ry]); self.grid_status[rx, ry] = True
    def update(self, sim_map, particle_sys):
        new_fire = []
        for fx, fy in self.fire_cells:
            if random.random() < FIRE_SPREAD_SPEED:
                nx, ny = fx+random.randint(-1, 1), fy+random.randint(-1, 1)
                if 0<=nx<sim_map.w and 0<=ny<sim_map.h:
                    if not sim_map.walls[nx, ny] and not self.grid_status[nx, ny]:
                        new_fire.append([nx, ny]); self.grid_status[nx, ny] = True
            particle_sys.add_smoke(fx, fy)
        self.fire_cells.extend(new_fire)
    def draw(self, screen, scale_x, scale_y):
        surf = pygame.Surface(screen.get_size(), pygame.SRCALPHA)
        for fx, fy in self.fire_cells:
            sx, sy = int(fx*scale_x), int(fy*scale_y)
            r = int(scale_x * 1.5 + random.uniform(-1,1))
            pygame.draw.circle(surf, (255, 100, 0, 100), (sx, sy), r)
            pygame.draw.circle(surf, (255, 255, 100, 200), (sx, sy), int(r*0.6))
        screen.blit(surf, (0,0), special_flags=pygame.BLEND_ADD)

class Agent:
    def __init__(self, x, y):
        self.pos = np.array([float(x), float(y)])
        self.speed = random.uniform(0.3, 0.6) 
        self.vel = np.array([0.0, 0.0])
        self.escaped = False
        self.panic = False
        self.dead = False
        self.stuck_timer = 0

    def update(self, sim_map, fire_mgr):
        if self.escaped or self.dead: return
        px, py = int(self.pos[0]), int(self.pos[1])

        # Status Checks
        if 0 <= px < sim_map.w and 0 <= py < sim_map.h:
            if fire_mgr.grid_status[px, py]: self.dead = True; return
        if sim_map.exits[px, py]: self.escaped = True; return

        # Movement Logic
        if self.panic:
            best = sim_map.flow_field[px, py]
            target = self.pos
            found = False
            for dx in [-1,0,1]:
                for dy in [-1,0,1]:
                    nx, ny = px+dx, py+dy
                    if 0<=nx<sim_map.w and 0<=ny<sim_map.h:
                        dist = sim_map.flow_field[nx, ny]
                        # Allow equal steps to cross flat areas
                        if dist <= best:
                            best = dist
                            target = np.array([nx + 0.5 + random.uniform(-0.1, 0.1), 
                                               ny + 0.5 + random.uniform(-0.1, 0.1)])
                            found = True
            if found:
                direction = target - self.pos
                norm = np.linalg.norm(direction)
                if norm > 0: self.vel = (direction/norm) * self.speed
            
            # Anti-Stuck Jolt
            if np.linalg.norm(self.vel) < 0.05: self.stuck_timer += 1
            else: self.stuck_timer = 0
            if self.stuck_timer > 15:
                self.vel = np.random.uniform(-1, 1, 2) * self.speed * 2
                self.stuck_timer = 0
        else:
            if random.random() < 0.05:
                self.vel = np.array([random.uniform(-0.5,0.5), random.uniform(-0.5,0.5)]) * 0.5

        # --- FIX 2: STRICT COLLISION CHECK ---
        # Look Ahead logic: Check where we WILL be, not where we ARE.
        proposed_pos = self.pos + self.vel
        npx, npy = int(proposed_pos[0]), int(proposed_pos[1])
        
        # 1. Bounds Check
        if not (0 <= npx < sim_map.w and 0 <= npy < sim_map.h):
            self.vel *= -1 # Turn back
            return

        # 2. Wall Check
        if not sim_map.walls[npx, npy]:
            # Safe to move
            self.pos = proposed_pos
        else:
            # 3. Collision Handling: Slide Logic
            # Try moving ONLY X
            slide_x = self.pos + np.array([self.vel[0], 0])
            sx, sy = int(slide_x[0]), int(slide_x[1])
            if not sim_map.walls[sx, sy]:
                self.pos = slide_x
                self.vel[1] *= -0.1 # Kill Y momentum
            else:
                # Try moving ONLY Y
                slide_y = self.pos + np.array([0, self.vel[1]])
                sx, sy = int(slide_y[0]), int(slide_y[1])
                if not sim_map.walls[sx, sy]:
                    self.pos = slide_y
                    self.vel[0] *= -0.1 # Kill X momentum
                else:
                    # Stuck in corner, bounce back
                    self.vel *= -0.5

def main():
    pygame.init()
    sim_map = SimulationMap()
    sim_map.load_and_setup()

    screen = pygame.display.set_mode((sim_map.screen_w, sim_map.screen_h))
    pygame.display.set_caption("Evacuation Simulator: Fixed Physics")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("Segoe UI", 24, bold=True)
    
    fire_mgr = FireManager()
    particles = ParticleSystem()
    scale_x = sim_map.screen_w / GRID_SIZE
    scale_y = sim_map.screen_h / GRID_SIZE

    # Spawn Agents (Ensure valid spawn)
    agents = []
    while len(agents) < NUM_AGENTS:
        rx, ry = random.randint(1, GRID_SIZE-2), random.randint(1, GRID_SIZE-2)
        # Check spawn validity strictly
        if not sim_map.walls[rx, ry] and not sim_map.exits[rx, ry]:
            # Also check neighbors to ensure not trapped inside a thick wall
            if not sim_map.walls[rx+1, ry] and not sim_map.walls[rx-1, ry]:
                agents.append(Agent(rx, ry))

    # UI State
    fire_active = False
    btn_rect = pygame.Rect(sim_map.screen_w // 2 - 75, 20, 150, 50)

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT: running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1 and btn_rect.collidepoint(event.pos):
                    fire_active = not fire_active
                    for a in agents: a.panic = fire_active
                    if fire_active: fire_mgr.ignite(sim_map)
                    else: 
                        fire_mgr.fire_cells = []; fire_mgr.grid_status.fill(False); particles.smoke = []

        if fire_active:
            fire_mgr.update(sim_map, particles)
            particles.update()

        for a in agents: a.update(sim_map, fire_mgr)

        screen.blit(sim_map.bg_surface, (0, 0))

        # Dead Agents
        for a in agents:
            if a.dead:
                sx, sy = int(a.pos[0]*scale_x), int(a.pos[1]*scale_y)
                pygame.draw.line(screen, COL_DEAD, (sx-3, sy-3), (sx+3, sy+3), 2)
                pygame.draw.line(screen, COL_DEAD, (sx+3, sy-3), (sx-3, sy+3), 2)

        # Live Agents
        for a in agents:
            if a.escaped or a.dead: continue
            col = COL_AGENT_PANIC if a.panic else COL_AGENT_SAFE
            sx, sy = int(a.pos[0]*scale_x), int(a.pos[1]*scale_y)
            pygame.draw.circle(screen, col, (sx, sy), AGENT_RADIUS)

        particles.draw(screen, scale_x, scale_y)
        if fire_active: fire_mgr.draw(screen, scale_x, scale_y)

        # Stats Overlay
        overlay = pygame.Surface((200, 90), pygame.SRCALPHA)
        overlay.fill((0, 0, 0, 180))
        screen.blit(overlay, (10, 10))
        
        escaped = sum(1 for a in agents if a.escaped)
        died = sum(1 for a in agents if a.dead)
        alive = NUM_AGENTS - escaped - died
        
        screen.blit(font.render(f"Escaped: {escaped}", True, (50, 255, 50)), (20, 20))
        screen.blit(font.render(f"Alive:   {alive}", True, (100, 200, 255)), (20, 45))
        screen.blit(font.render(f"Casualties: {died}", True, (255, 50, 50)), (20, 70))

        btn_col = (220, 60, 60) if fire_active else (60, 220, 60)
        pygame.draw.rect(screen, btn_col, btn_rect, border_radius=8)
        pygame.draw.rect(screen, (255,255,255), btn_rect, 2, border_radius=8)
        
        lbl = "EVACUATE!" if not fire_active else "RESET"
        txt = font.render(lbl, True, (255,255,255))
        txt_rect = txt.get_rect(center=btn_rect.center)
        screen.blit(txt, txt_rect)

        pygame.display.flip()
        clock.tick(FPS)

    pygame.quit()

if __name__ == "__main__":
    main()

Thickening walls...
Generating navigation mesh...
Ready.


Walls fixed

In [43]:
import pygame
import numpy as np
import random
from collections import deque
from scipy.ndimage import binary_dilation

# --- CONFIGURATION ---
MAX_WINDOW_SIZE = 1000
GRID_SIZE = 150   # INCREASED RESOLUTION (Helps detect thin walls)
FPS = 60

# Simulation Settings
NUM_AGENTS = 180
AGENT_RADIUS = 3
FIRE_SPREAD_SPEED = 0.05
FIRE_KILL_RADIUS = 5

# --- FILE PATHS ---
# MAKE SURE THESE ARE CORRECT
WALL_IMG_PATH = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_walls.png"
OBST_IMG_PATH = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_obstacles.png"
EXIT_IMG_PATH = r"C:\Users\joshu\Downloads\Fourth Plant Draft Seperate_exit.png"

# --- COLORS ---
COL_AGENT_SAFE = (0, 120, 255)
COL_AGENT_PANIC = (255, 50, 50)
COL_SMOKE = (50, 50, 50)
COL_DEAD = (20, 20, 20)
COL_DEBUG_WALL = (0, 0, 0) # Black for debug overlay

class SimulationMap:
    def __init__(self):
        self.w = GRID_SIZE
        self.h = GRID_SIZE
        self.walls = None
        self.exits = None
        self.flow_field = None
        self.screen_w = 0
        self.screen_h = 0
        self.bg_surface = None

    def load_and_setup(self):
        try:
            # 1. Load Images
            raw_wall = pygame.image.load(WALL_IMG_PATH)
            raw_obst = pygame.image.load(OBST_IMG_PATH)
            raw_exit = pygame.image.load(EXIT_IMG_PATH)

            # 2. Setup Screen Size
            img_w, img_h = raw_wall.get_size()
            scale = min(MAX_WINDOW_SIZE / img_w, MAX_WINDOW_SIZE / img_h)
            self.screen_w = int(img_w * scale)
            self.screen_h = int(img_h * scale)

            # 3. Visual Background
            self.bg_surface = pygame.Surface((self.screen_w, self.screen_h))
            self.bg_surface.fill((255, 255, 255)) 
            
            vis_wall = pygame.transform.scale(raw_wall, (self.screen_w, self.screen_h))
            vis_obst = pygame.transform.scale(raw_obst, (self.screen_w, self.screen_h))
            vis_exit = pygame.transform.scale(raw_exit, (self.screen_w, self.screen_h))
            
            # Blit (Stacking them)
            # Assuming white backgrounds, we use them as transparent
            vis_obst.set_colorkey((255,255,255)); self.bg_surface.blit(vis_obst, (0,0))
            vis_wall.set_colorkey((255,255,255)); self.bg_surface.blit(vis_wall, (0,0))
            vis_exit.set_colorkey((255,255,255)); self.bg_surface.blit(vis_exit, (0,0))

            # 4. Logic Processing
            log_w = pygame.transform.scale(raw_wall, (self.w, self.h))
            log_o = pygame.transform.scale(raw_obst, (self.w, self.h))
            log_e = pygame.transform.scale(raw_exit, (self.w, self.h))

            w_arr = pygame.surfarray.array3d(log_w)
            o_arr = pygame.surfarray.array3d(log_o)
            e_arr = pygame.surfarray.array3d(log_e)

            # --- CRITICAL FIX: BETTER DETECTION LOGIC ---
            # Instead of looking for "Dark" pixels, we look for "Not White" pixels.
            # This catches light grey walls that were being ignored.
            # 250 is very close to white (255). Anything darker than 250 is a wall.
            self.walls = np.mean(w_arr, axis=2) < 250 
            self.obstacles = np.mean(o_arr, axis=2) < 250
            self.exits = np.mean(e_arr, axis=2) < 250
            
            # Combine Walls + Obstacles
            self.barriers = np.logical_or(self.walls, self.obstacles)
            
            # Thicken Logic Walls (Safety Margin)
            # This expands the walls by 1 block so agents don't clip through thin lines
            self.barriers = binary_dilation(self.barriers, iterations=1)
            
            # Clean Exits (Exits are never barriers)
            self.barriers[self.exits] = False
            
            print("Navigation Mesh Generating...")
            self._generate_flow_field()
            print("Ready. PRESS 'D' TO TOGGLE DEBUG VIEW.")

        except Exception as e:
            print(f"Error: {e}")
            exit()

    def _generate_flow_field(self):
        self.flow_field = np.full((self.w, self.h), 9999)
        queue = deque()
        for x in range(self.w):
            for y in range(self.h):
                if self.exits[x, y]:
                    self.flow_field[x, y] = 0
                    queue.append((x, y))

        while queue:
            x, y = queue.popleft()
            val = self.flow_field[x, y]
            neighbors = [(x+1,y),(x-1,y),(x,y+1),(x,y-1),(x+1,y+1),(x-1,y-1),(x+1,y-1),(x-1,y+1)]
            for nx, ny in neighbors:
                if 0<=nx<self.w and 0<=ny<self.h:
                    if not self.barriers[nx, ny]:
                        cost = 1 if (nx==x or ny==y) else 1.4
                        if self.flow_field[nx, ny] > val + cost:
                            self.flow_field[nx, ny] = val + cost
                            queue.append((nx, ny))
    
    def is_valid_spawn(self, x, y):
        # Strict spawn check
        if self.barriers[x, y] or self.exits[x, y]: return False
        # Check neighbors (buffer)
        if (x>0 and self.barriers[x-1,y]) or (x<self.w-1 and self.barriers[x+1,y]): return False
        if (y>0 and self.barriers[x,y-1]) or (y<self.h-1 and self.barriers[x,y+1]): return False
        return True

# --- SYSTEMS ---
class ParticleSystem:
    def __init__(self): self.smoke = [] 
    def add_smoke(self, x, y):
        if random.random() < 0.3:
            self.smoke.append([x + random.uniform(-1,1), y + random.uniform(-1,1), random.uniform(2,5), 160])
    def update(self):
        for s in self.smoke: s[1]-=0.1; s[2]+=0.05; s[3]-=2
        self.smoke = [s for s in self.smoke if s[3]>0]
    def draw(self, screen, sx, sy):
        surf = pygame.Surface(screen.get_size(), pygame.SRCALPHA)
        for s in self.smoke: pygame.draw.circle(surf, (*COL_SMOKE, int(s[3])), (int(s[0]*sx), int(s[1]*sy)), int(s[2]*sx))
        screen.blit(surf, (0,0))

class FireManager:
    def __init__(self):
        self.fire_cells = []; self.grid_status = np.zeros((GRID_SIZE, GRID_SIZE), dtype=bool) 
    def ignite(self, sim_map):
        self.fire_cells = []; self.grid_status.fill(False)
        for _ in range(3):
            rx, ry = random.randint(1, sim_map.w-2), random.randint(1, sim_map.h-2)
            if not sim_map.barriers[rx, ry]:
                self.fire_cells.append([rx, ry]); self.grid_status[rx, ry] = True
    def update(self, sim_map, particle_sys):
        new_fire = []
        for fx, fy in self.fire_cells:
            if random.random() < FIRE_SPREAD_SPEED:
                nx, ny = fx+random.randint(-1,1), fy+random.randint(-1,1)
                if 0<=nx<sim_map.w and 0<=ny<sim_map.h:
                    if not sim_map.barriers[nx, ny] and not self.grid_status[nx, ny]:
                        new_fire.append([nx, ny]); self.grid_status[nx, ny] = True
            particle_sys.add_smoke(fx, fy)
        self.fire_cells.extend(new_fire)
    def draw(self, screen, sx, sy):
        surf = pygame.Surface(screen.get_size(), pygame.SRCALPHA)
        for fx, fy in self.fire_cells:
            px, py = int(fx*sx), int(fy*sy)
            r = int(sx * 1.5 + random.uniform(-1,1))
            pygame.draw.circle(surf, (255, 100, 0, 100), (px, py), r)
            pygame.draw.circle(surf, (255, 255, 100, 200), (px, py), int(r*0.6))
        screen.blit(surf, (0,0), special_flags=pygame.BLEND_ADD)

class Agent:
    def __init__(self, x, y):
        self.pos = np.array([float(x), float(y)])
        self.speed = random.uniform(0.3, 0.6)
        self.vel = np.array([0.0, 0.0])
        self.escaped = False; self.panic = False; self.dead = False; self.stuck_timer = 0

    def update(self, sim_map, fire_mgr):
        if self.escaped or self.dead: return
        px, py = int(self.pos[0]), int(self.pos[1])

        # Status Check
        if fire_mgr.grid_status[px, py]: self.dead = True; return
        if sim_map.exits[px, py]: self.escaped = True; return

        # Movement
        if self.panic:
            best = sim_map.flow_field[px, py]
            target = self.pos; found = False
            for dx in [-1,0,1]:
                for dy in [-1,0,1]:
                    nx, ny = px+dx, py+dy
                    if 0<=nx<sim_map.w and 0<=ny<sim_map.h:
                        dist = sim_map.flow_field[nx, ny]
                        if dist <= best:
                            best = dist
                            target = np.array([nx+0.5, ny+0.5]) + np.random.uniform(-0.1,0.1,2)
                            found = True
            if found:
                direction = target - self.pos
                norm = np.linalg.norm(direction)
                if norm > 0: self.vel = (direction/norm) * self.speed
            
            if np.linalg.norm(self.vel) < 0.05: self.stuck_timer += 1
            else: self.stuck_timer = 0
            if self.stuck_timer > 15:
                self.vel = np.random.uniform(-1,1,2)*self.speed*2; self.stuck_timer=0
        else:
            if random.random() < 0.05:
                self.vel = np.random.uniform(-0.3,0.3,2)

        # Strict Collision
        proposed = self.pos + self.vel
        npx, npy = int(proposed[0]), int(proposed[1])
        
        # Check Bounds
        if not (0<=npx<sim_map.w and 0<=npy<sim_map.h):
            self.vel *= -1; return

        # Check Wall
        if not sim_map.barriers[npx, npy]:
            self.pos = proposed
        else:
            # Slide X
            slide_x = self.pos + np.array([self.vel[0], 0])
            sx, sy = int(slide_x[0]), int(slide_x[1])
            if not sim_map.barriers[sx, sy]:
                self.pos = slide_x; self.vel[1] *= -0.1
            else:
                # Slide Y
                slide_y = self.pos + np.array([0, self.vel[1]])
                sx, sy = int(slide_y[0]), int(slide_y[1])
                if not sim_map.barriers[sx, sy]:
                    self.pos = slide_y; self.vel[0] *= -0.1
                else:
                    self.vel *= -0.5

def main():
    pygame.init()
    sim_map = SimulationMap()
    sim_map.load_and_setup()

    screen = pygame.display.set_mode((sim_map.screen_w, sim_map.screen_h))
    pygame.display.set_caption("Sim: Press 'D' for Debug Mode")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("Segoe UI", 24, bold=True)
    
    fire_mgr = FireManager()
    particles = ParticleSystem()
    scale_x = sim_map.screen_w / GRID_SIZE
    scale_y = sim_map.screen_h / GRID_SIZE

    print("Spawning...")
    agents = []
    while len(agents) < NUM_AGENTS:
        rx, ry = random.randint(1, GRID_SIZE-2), random.randint(1, GRID_SIZE-2)
        if sim_map.is_valid_spawn(rx, ry):
            agents.append(Agent(rx, ry))

    btn_rect = pygame.Rect(sim_map.screen_w//2 - 75, 20, 150, 50)
    fire_active = False
    debug_mode = False # TOGGLE THIS WITH 'D'

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT: running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_d:
                    debug_mode = not debug_mode
                    print(f"Debug Mode: {debug_mode}")
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1 and btn_rect.collidepoint(event.pos):
                    fire_active = not fire_active
                    for a in agents: a.panic = fire_active
                    if fire_active: fire_mgr.ignite(sim_map)
                    else: fire_mgr.fire_cells = []; fire_mgr.grid_status.fill(False); particles.smoke = []

        if fire_active:
            fire_mgr.update(sim_map, particles)
            particles.update()
        for a in agents: a.update(sim_map, fire_mgr)

        screen.blit(sim_map.bg_surface, (0, 0))

        # --- DEBUG OVERLAY ---
        if debug_mode:
            # Draws black boxes where the computer thinks walls are
            for x in range(GRID_SIZE):
                for y in range(GRID_SIZE):
                    if sim_map.barriers[x, y]:
                        rect = (x * scale_x, y * scale_y, scale_x + 1, scale_y + 1)
                        pygame.draw.rect(screen, COL_DEBUG_WALL, rect)
        
        # Agents
        for a in agents:
            if a.dead:
                sx, sy = int(a.pos[0]*scale_x), int(a.pos[1]*scale_y)
                pygame.draw.line(screen, COL_DEAD, (sx-3, sy-3), (sx+3, sy+3), 2)
                pygame.draw.line(screen, COL_DEAD, (sx+3, sy-3), (sx-3, sy+3), 2)
        
        for a in agents:
            if a.escaped or a.dead: continue
            col = COL_AGENT_PANIC if a.panic else COL_AGENT_SAFE
            sx, sy = int(a.pos[0]*scale_x), int(a.pos[1]*scale_y)
            pygame.draw.circle(screen, col, (sx, sy), AGENT_RADIUS)

        particles.draw(screen, scale_x, scale_y)
        if fire_active: fire_mgr.draw(screen, scale_x, scale_y)

        # UI
        stats_bg = pygame.Surface((220, 90), pygame.SRCALPHA); stats_bg.fill((0,0,0,180))
        screen.blit(stats_bg, (10, 10))
        esc = sum(1 for a in agents if a.escaped); died = sum(1 for a in agents if a.dead)
        screen.blit(font.render(f"Escaped: {esc}", True, (50, 255, 50)), (20, 20))
        screen.blit(font.render(f"Alive:   {NUM_AGENTS-esc-died}", True, (100, 200, 255)), (20, 45))
        
        col = (220, 60, 60) if fire_active else (60, 220, 60)
        pygame.draw.rect(screen, col, btn_rect, border_radius=8)
        lbl = "RESET" if fire_active else "FIRE"
        screen.blit(font.render(lbl, True, (255,255,255)), (btn_rect.x+40, btn_rect.y+10))

        pygame.display.flip()
        clock.tick(FPS)

    pygame.quit()

if __name__ == "__main__":
    main()

Navigation Mesh Generating...
Ready. PRESS 'D' TO TOGGLE DEBUG VIEW.
Spawning...
